# Loading data

In [ ]:
# TODO
# Create columns for day, hour, and week
# Resample data
# create method that generates a graph, passing arguments the time window required

In [1]:
# Importing dependencies
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import calendar #necessary to convert a timestamp into a date
#from datetime import datetime
import time
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

/Users/louis/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/louis/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# Data preprocessing

In [ ]:
%reload_ext autoreload
%autoreload 2
import processing_louis

In [ ]:
# run the processing data methode
%run '/Users/louis/Documents/Snips/snips_test/processing_louis.py'
data = processing_data()

In [ ]:
# Preprocess data


In [ ]:
# Range of times
a = data['last_update'].max() - data['last_update'].min()
b = (a/(1000*3600*24))
print 'Conlusion ---> The time span of our dataset is %s' %b

In [ ]:
max_date = data['last_update'].max()
min_date = data['last_update'].min()
print 'max date ----> %s' %max_date
print 'min date ----> %s' %min_date

# Available_bike_stands/bike_stands/available_bike

In [ ]:
# Overview of min/max of the three variables per station
data.groupby(['address']).agg({'available_bike_stands':[np.min,np.max], 'bike_stands': [np.min, np.max], 'available_bikes': [np.min, np.max]  })

In [ ]:
# Histogram of number of bike_stands per station
#1st solution
#data.groupby(['address']).agg({'bike_stands': np.max}).sort_index(ascending=False).plot(kind='barh')
#2nd and simpler solution (though we do not have the labels
data['bike_stands'].value_counts(dropna = False).plot(kind ='barh')



In [ ]:
delta.value_counts(dropna = False).plot(kind = 'barh')

# Status

In [ ]:
# Are there some stations that never close?
data.groupby(['address', 'status']).count().iloc[:,1]
print 'We have much more data points when stations are open'

In [ ]:
pivot_hour =data['hour'].value_counts(dropna = False).sort_index(ascending=False).plot(kind='barh')
print pivot_hour

In [ ]:
# Are there stations that do not have all hours covered?
pivot_frequ = data.groupby(['address','hour']).count().iloc[:,1]
pivot_frequ = pivot_frequ.reset_index(level=1)
pivot_frequ = pivot_frequ.reset_index(level = 0)
pivot_frequ.head()


In [ ]:
# Focus one a given station
axis_x = pivot_frequ[pivot_frequ['address'] == "1 AVENUE DE LA MOTTE PICQUET - 75007 PARIS"].loc[:,'hour']
axis_y = pivot_frequ[pivot_frequ['address'] == "1 AVENUE DE LA MOTTE PICQUET - 75007 PARIS"].loc[:,'available_bikes']

plt.plot(axis_x, axis_y)

In [ ]:
# Plot all stations
plt.plot(pivot_frequ['hour'], pivot_frequ['available_bikes'])
print 'conclusion --> we have much more datapoints during the day, should we adjust for it?'

In [ ]:
pivot_frequ[pivot_frequ['address'] == "1 AVENUE DE LA MOTTE PICQUET - 75007 PARIS"].plot(x = 'hour', y = 'available_bikes')

In [ ]:
# If we want a line by address we need to have address as column labels
pivot_frequ.head()
pivot_frequ_bis = pd.pivot_table(pivot_frequ, index ='hour',columns = ['address'], values = 'available_bikes', aggfunc = np.sum)


In [ ]:
pivot_frequ_bis.plot(legend = False)

In [ ]:
# What is the pattern for status?  --> plot count of OPEN and CLOSE in function of the hour

# Make the count
piv_frequ = data.groupby(['address', 'hour', 'status']).count()
# Pass index into columns so we can perform the pivot
piv_frequ = piv_frequ.reset_index(level=[0,1,2])
#pivot_frequ = pivot_frequ.reset_index(level = 0)
piv2 = pd.pivot_table(piv_frequ, columns =['status'], index = ['hour'], values = ['available_bikes'], aggfunc = np.sum)
# plot
piv2.plot()

# Patterns in bikes availables (over a day)

In [ ]:
# Plot the average number of bikes available in function of hour
piv = pd.pivot_table(data, index = 'hour', columns = 'address', values = 'available_bikes', aggfunc = np.mean)
piv.plot(legend = False)

In [ ]:
piv_2 = pd.pivot_table(data,index = 'hour', columns = 'address', values = 'av_ratio', aggfunc = np.mean)
piv_2.plot(legend = False)

In [ ]:
# Create mean with address and arrondissement as index
piv_3 = pd.pivot_table(data, index = ['hour'], columns = ['arrondissements'], values = 'av_ratio', aggfunc = np.mean)
piv_3.head()

In [ ]:
# pass arrondissements as column
piv_3.plot()


# Patterns in bikes available  (over a week)

In [ ]:
data.columns.values

In [ ]:
# bring last update as a column rather than index
#data.reset_index(level=0, inplace=True)

In [ ]:
# create pivot table of the desired value_key
def timeseries_table(data,value_key):
    table = data.pivot_table(index=['date'], columns=['arrondissements'], values=value_key,aggfunc= np.min)
    return table

# sample data
def sampling(data, data_n, delta):
    # resampling both tables
    data_s = data.resample(str(delta) + 'Min').mean().ffill()
    # normalizing data by data_n
    data_n_s = data.resample(str(delta) +'Min'). max().ffill()
    table = data_s.div(data_n_s)*100
    return table

# plot selected stations
def plot_stations(input_data, start_date, end_date, arrondissements):
    data_filt = input_data.ix[start_date:end_date,:]
    data_filt = data_filt.loc[:,arrondissements]
    data_filt.plot(figsize=(20,8),legend = False)

In [ ]:
# Declare start_date and end_date
start_date  = datetime.datetime(2015,11,10,0,0,0)
end_date  =  datetime.datetime(2015,11,18,0,0,0)
arrondissements = [1]

# create pivot tables of numerator and denominator
av_bikes_table = timeseries_table(data, 'available_bikes')
tot_bikes_table = timeseries_table(data, 'tot_bikes')

# resample and normalize
normalized_av_bikes = sampling(av_bikes_table, tot_bikes_table, 10)
plot_stations(normalized_av_bikes, start_date, end_date, arrondissements)

In [ ]:
# sample both tables and return sampled ratio table
normalized_av_bikes = sampling(av_bikes_table, tot_bikes_table, 60)
normalized_av_bikes.head()

# Process and return data ready for modeling

In [ ]:
# TODO
# --> resample only on values that need to be resampled (put date and name/address as index, and create a transition table to find all the
# fixed info

In [2]:
# load and preprocess snips data
%reload_ext autoreload
%autoreload 2

%run '/Users/louis/Documents/Snips/snips_test/processing_louis.py'
data = processing_data()

KeyboardInterrupt: 

In [ ]:
# load meteo data + clean + dummify
%run '/Users/louis/Documents/Snips/snips_test/meteo_louis.py'
data_meteo = get_meteo()
data_meteo.head()

In [ ]:
# Create data ready for modeling
%run '/Users/louis/Documents/Snips/snips_test/process_data_model.py'
data_model = data_model(data, data_meteo, 30, 'av_ratio')

In [ ]:
data_model.columns.values

# Linear regression

In [ ]:
from sklearn import metrics
from sklearn import linear_model,cross_validation
from sklearn.cross_validation import train_test_split

In [ ]:
# create training and test sets
train, test = train_test_split(data_model, test_size = 0.2)
# Define y
y_train = train['av_ratio']
y_test = test['av_ratio']
# Define X
del train['av_ratio']
del test['av_ratio']

x_train = train
x_test = test

x_train.head()

# declare linear regression
#lr = linear_model.LogisticRegression(C = 0.1)
# train
#lr.fit(x_train, y_train)

# predict on training set + on cv set
#y_train_predict = lr.predict(x_train)
#y_test_predict = lr.predict(x_test)


In [ ]:
x_train.columns.values